In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.3 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd



# Create the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
# model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-bert-base-dot-v5")
model = AutoModelForQuestionAnswering.from_pretrained("sentence-transformers/msmarco-bert-base-dot-v5")

# Define the question answering pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, batch_size = 1)



Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at sentence-transformers/msmarco-bert-base-dot-v5 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load the dataset
df = pd.read_csv("multi-train.csv")

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import pandas as pd
df_validation = pd.read_csv("multi-valid.csv")

In [ ]:
df

,Unnamed: 0,uuid,postText,targetParagraphs,targetTitle,spoiler,spoilerPositions,tags,start_positions,end_positions
1,1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,NASA sets date for full recovery of ozone hole,2070 is shaping up to be a great year for Moth...,Hole In Ozone Layer Expected To Make Full Reco...,2070,"[[[0, 0], [0, 4]]]",phrase,[0],[4]
2,2,008b7b19-0445-4e16-8f9e-075b73f80ca4,This is what makes employees happy -- and it's...,"Despite common belief, money isn't the key to ...",Intellectual Stimulation Trumps Money For Empl...,intellectual stimulation,"[[[1, 186], [1, 210]]]",phrase,[272],[296]
4,4,31b108a3-c828-421a-a4b9-cf651e9ac859,The perfect way to cook rice so that it's perf...,"Boiling rice may seem simple, but there is a v...",Revealed: The perfect way to cook rice so that...,in a rice cooker,"[[[5, 60], [5, 76]]]",phrase,[655],[671]
10,10,46aa8d72-fbd4-4796-a85b-0a0363c21812,Analysis: This may be the most brutal number i...,Plenty has been made of the big Congressional ...,This may be the most brutal number in the CBO ...,750 percent,"[[[2, 109], [2, 120]]]",phrase,[583],[594]
11,11,917e1106-413c-4be5-818b-ad500314feaa,#TeenMom2 star @PBandJenelley_1 reveals the se...,"""Teen Mom 2"" star Jenelle Evans took to Twitte...",'Teen Mom 2' Star Jenelle Evans Reveals Sex Of...,boy,"[[[0, 103], [0, 106]]]",phrase,[103],[106]
...,...,...,...,...,...,...,...,...,...,...
3178,3178,edec6cde-bbd3-4f50-ba51-ef98bdf19b63,"He Dug A Huge Hole In His Backyard, And What H...","When it comes to home projects, sometimes the ...","He Dug A Huge Hole In His Backyard, And What H...",own underground bunker!,"[[[20, 70], [20, 93]]]",phrase,[6104],[6127]
3179,3179,84277c0f-4ef1-444e-8e04-55c84af593e5,Best Buy Has An Insane Xbox One Deal For A Lim...,Best Buy Has An Insane Xbox One Deal For A Lim...,Best Buy Has An Insane Xbox One Deal For A Lim...,$50 off,"[[[8, 101], [8, 108]]]",phrase,[1814],[1821]
3183,3183,5b61d712-8b03-4ee6-ba04-b39ce2b206f7,Student forced to carry papers to prove she ca...,She's not a tourist visiting a foreign country...,"Melona Clark, Hampton University Student, Carr...",student at Hampton University in Virginia,"[[[1, 9], [1, 50]]]",phrase,[163],[204]
3198,3198,9d05984c-3920-47c0-aa97-8df58cca1fec,You need to see this Twitter account that pred...,What the HELL?! 1. Unless you’re somewhere wit...,"WTF, It Looks Like This Twitter Account ""Predi...",@beyoncefan666,"[[[3, 55], [3, 69]]]",phrase,[408],[422]


In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["postText"]]
    inputs = tokenizer(
        questions,
        examples["targetParagraphs"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["spoiler"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        start_char = int(examples["start_positions"][i][1])
        end_char = start_char + len(examples["spoiler"][i])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [ ]:
from datasets import load_dataset
import datasets

val_dataset2 = datasets.Dataset.from_pandas(df_test)
train_dataset2 = datasets.Dataset.from_pandas(df_train)

In [ ]:
train_dataset = train_dataset2.map(preprocess_function, batched=True,
    remove_columns=train_dataset2.column_names)


Map:   0%|          | 0/1093 [00:00<?, ? examples/s]

In [ ]:
validation_dataset = val_dataset2.map(preprocess_function, batched=True,
    remove_columns=val_dataset2.column_names)


Map:   0%|          | 0/274 [00:00<?, ? examples/s]

In [ ]:
print(validation_dataset)

Dataset({
    features: ['start_positions', 'end_positions', 'input_ids', 'attention_mask'],
    num_rows: 274
})


In [ ]:
print(train_dataset)

Dataset({
    features: ['start_positions', 'end_positions', 'input_ids', 'attention_mask'],
    num_rows: 1093
})


In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!pip install transformers --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="qa_train_passage",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    #push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.959400,1.619157
2,1.429000,1.524606
3,1.121600,1.678660
4,0.869200,2.018714
5,0.604700,2.321848


TrainOutput(global_step=2735, training_loss=1.1401159366698326, metrics={'train_runtime': 591.8352, 'train_samples_per_second': 9.234, 'train_steps_per_second': 4.621, 'total_flos': 1070990081671680.0, 'train_loss': 1.1401159366698326, 'epoch': 5.0})

In [ ]:
# Loop through each row in the dataset and train the model
for index, row in df.iterrows():
    context = row["targetParagraphs"]
    question = row["targetTitle"]
    if isinstance(row["spoiler"], str):
      answer = row["spoiler"].split('.')
    else: 
      answer = ""
    #answer = row["spoiler"].split('.')
    qa_pipeline(context=context, question=question, answer=answer)

# # Use the trained model to generate responses
# prompt = "What is the answer to the following question?"
# question = "What is the capital of France?"
# context = "France is a country located in Western Europe. Its capital is Paris."

# response = qa_pipeline(context=context, question=question)

# print(response["answer"])

In [ ]:
spoilers = []

for index, row in df_validation.iterrows():
  curr = []
  for val in str(row['targetParagraphs']).split('\n'):
    # print(index, val)
    if val!="" and not val.isdigit():
      context = val
      question = row["targetTitle"]
      response = qa_pipeline(context=context, question=question)
      # adding a threshold limit to the probability score
      if response["score"] >= 0.01
        curr.append(response["answer"])
      print(response)
  print(curr)


  spoilers.append(curr)



{'score': 0.00029325985815376043, 'start': 16, 'end': 31, 'answer': 'Segall Maricopa'}
{'score': 0.00014652364188805223, 'start': 346, 'end': 375, 'answer': 'sheriff of Arizona’s Maricopa'}
{'score': 0.000800741312559694, 'start': 164, 'end': 168, 'answer': 'were'}
{'score': 0.0050505357794463634, 'start': 27, 'end': 63, 'answer': 'had unusually harsh words for Arpaio'}
{'score': 0.0011482243426144123, 'start': 38, 'end': 75, 'answer': 'inequitable application of discipline'}
{'score': 0.0004309076757635921, 'start': 105, 'end': 119, 'answer': 'had encouraged'}
{'score': 0.0005470547475852072, 'start': 159, 'end': 195, 'answer': 'had violated the order "out of spite'}
{'score': 0.0035541693214327097, 'start': 39, 'end': 73, 'answer': 'had violated the order unknowingly'}
{'score': 0.0005591019289568067, 'start': 106, 'end': 166, 'answer': 'contempt on two counts, and retired Chief Brian Sands and Lt'}
{'score': 0.00042490652413107455, 'start': 153, 'end': 254, 'answer': 'evidentiary he

In [ ]:
print(spoilers)

[['Segall Maricopa', 'sheriff of Arizona’s Maricopa', 'were', 'had unusually harsh words for Arpaio', 'inequitable application of discipline', 'had encouraged', 'had violated the order "out of spite', 'had violated the order unknowingly', 'contempt on two counts, and retired Chief Brian Sands and Lt', 'evidentiary hearing, Sheriff Arpaio and Chief Deputy Sheridan made multiple intentional misstatements', 'wither', 'investigator to look into comments Snow’s wife purportedly made in a restaurant', 'impartial and should recuse', 'disobeyed orders to gather evidence and continued', 'thousands of pieces of evidence from the plaintiffs and deleted', 'the sheriff to resign', 'be entrusted', 'the original case against Arpaio, demanded', 'investigations that root out and punish misconduct', 'was one'], ['less, to one singular staple', 'Iggy Pop, Johnny', "CBGB's legacy", 'are you up', "'m just here", 'will start [in', 'Are you more of a theater guy', 'was just, yeah, I’m enjoying', 'Were you fa

In [ ]:
final_spoilers = []
final_spoilers = [' '.join(i) for i in spoilers]
final_spoilers

['Segall Maricopa sheriff of Arizona’s Maricopa were had unusually harsh words for Arpaio inequitable application of discipline had encouraged had violated the order "out of spite had violated the order unknowingly contempt on two counts, and retired Chief Brian Sands and Lt evidentiary hearing, Sheriff Arpaio and Chief Deputy Sheridan made multiple intentional misstatements wither investigator to look into comments Snow’s wife purportedly made in a restaurant impartial and should recuse disobeyed orders to gather evidence and continued thousands of pieces of evidence from the plaintiffs and deleted the sheriff to resign be entrusted the original case against Arpaio, demanded investigations that root out and punish misconduct was one',
 "less, to one singular staple Iggy Pop, Johnny CBGB's legacy are you up 'm just here will start [in Are you more of a theater guy was just, yeah, I’m enjoying Were you familiar with the legacy very much was amazing. They just had so much Did you grow up

In [ ]:
df_validation['generated_spoiler'] = final_spoilers

In [ ]:
import pandas as pd
from nltk.tokenize import TreebankWordTokenizer

# Define the tokenizer function
tokenizer = TreebankWordTokenizer().tokenize

# Tokenize the generated spoilers
df_validation['tokenized_generated_spoiler'] = df_validation['generated_spoiler'].apply(tokenizer)
df_validation['tokenized_spoiler'] = df_validation['spoiler'].apply(tokenizer)




In [ ]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from bert_score import score

total_precision, total_recall, total_f1, count = 0, 0, 0, 0

for index, row in df_validation.iterrows():
    if row['tags'] == "multi":
        # Calculate the BERT score for each pair of sentences
        scores = score([row['generated_spoiler']], [row['spoiler']], lang='en', verbose=False)

        # Extract precision, recall, and F1 scores
        precision, recall, f1 = scores

        # Add to running total
        total_precision += precision.mean()
        total_recall += recall.mean()
        total_f1 += f1.mean()
        count += 1

if count > 0:
    # Calculate and print average scores
    avg_precision = total_precision / count
    avg_recall = total_recall / count
    avg_f1 = total_f1 / count
    print(f"BERT precision score (average): {avg_precision:.4f}")
    print(f"BERT recall score (average): {avg_recall:.4f}")
    print(f"BERT F1 score (average): {avg_f1:.4f}")
else:
    print("No rows with 'tags' equal to 'phrase' found in DataFrame")


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaM

BERT precision score (average): 0.7946
BERT recall score (average): 0.8352
BERT F1 score (average): 0.8141


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
from nltk.translate import meteor_score

meteor_scores = []

for index, row in df_validation.iterrows():
    if row['tags'] == "multi":
        ref = row['spoiler']
        hyp = row['generated_spoiler']

        ref_tokens = nltk.word_tokenize(ref)
        hyp_tokens = nltk.word_tokenize(hyp)

        score = meteor_score.meteor_score([ref_tokens], hyp_tokens)
        meteor_scores.append(score)

average_meteor_score = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR score: {average_meteor_score}")

Average METEOR score: 0.22928763380649475


In [ ]:
import nltk

bleu_scores = []

for index, row in df_validation.iterrows():
    if row['tags'] == "multi":
        true_spoiler = row['tokenized_spoiler']
        myoutput = row['tokenized_generated_spoiler']
        bleu_score = nltk.translate.bleu_score.sentence_bleu([true_spoiler], myoutput)
        bleu_scores.append(bleu_score)

average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score for multi: {average_bleu_score}")

Average BLEU score for multi: 0.04048916814793454


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
print(df_validation[['spoiler','generated_spoiler']])